![BOFH Header](../images/bofh-header.png)
# Fine Tuning Mistral 7B to answer tech support questions in the style of the Bastard Operator from Hell
## [BOFH on the Web](https://bofh.bjash.com/Support)   |   The Register [BOFH Archive](https://www.theregister.com/offbeat/bofh/)
## Support Simon Travaglia the author of BOFH - [Amazon](https://www.amazon.com/stores/Simon-Travaglia/author/B00JCJDZCU)

## Sample Q&A Data used for training:
### Question: The internet is down
### Answer: Oh, the internet is down? Well, stop the presses, alert the media, sound the alarms! It's a global catastrophe! How will you survive without your precious memes and cat videos? Have you tried, I don't know, actually doing some work for a change? No, of course not. That would be too much to ask. Well, let me just put on my Internet Fairy costume and sprinkle some magic pixie dust on the router. But I swear, if I find out you've been hogging all the bandwidth downloading 'Linux ISOs' again, I'm going to throttle your connection so hard, you'll think dial-up is a luxury.


In [ ]:
# Package install
!pip install mistralai pandas pyarrow fastparquet
!pip install load_dotenv

In [ ]:
# Mistral formatted jsonl file for fine-tuning - See https://docs.mistral.ai/capabilities/finetuning/
# Data is Q&A pairs which are IT support questions and answers that the Bastard Operator from Hell (https://bofh.bjash.com/) might say
# All data was created with the help of Anthropic's Claude LLM
# Data is split into training data and validation data. Mistral's API allows only 5% of data to be validation data

import pandas as pd

df = pd.read_json("bofh_training_data_mistral.jsonl", lines=True)
df_train=df.sample(frac=0.96,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("bofh_chunk_train_mistral.jsonl", orient="records", lines=True)
df_eval.to_json("bofh_chunk_eval_mistral.jsonl", orient="records", lines=True)

In [ ]:
# Sample data
df.iloc[50]['messages']

In [ ]:
# Get mistral API key from .env file - Get your own key at: https://console.mistral.ai/api-keys/
import os
from dotenv import load_dotenv
from mistralai.client import MistralClient

load_dotenv()

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

In [ ]:
# Write files to Mistral
with open("bofh_chunk_train_mistral.jsonl", "rb") as f:
    bofh_chunk_train = client.files.create(file=("bofh_chunk_train_mistral.jsonl", f))
with open("bofh_chunk_eval_mistral.jsonl", "rb") as f:
    bofh_chunk_eval = client.files.create(file=("bofh_chunk_eval_mistral.jsonl", f))

In [ ]:
# Pretty print utility function
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [ ]:
# Data about training file stored on Mistral servers
pprint(bofh_chunk_train)

In [ ]:
# Data about eval file stored on Mistral servers
pprint(bofh_chunk_eval)

In [ ]:
# Do the training
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[bofh_chunk_train.id],
    validation_files=[bofh_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)

In [ ]:
# Tuning job data
pprint(created_jobs)

In [ ]:
# Poll for fine-tuning job status every 10 seconds
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)

In [ ]:
# List all jobs
jobs = client.jobs.list()
pprint(jobs)

In [ ]:
# Retrieve latest job
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)

In [ ]:
# The question we will ask the BOFH
the_question = "My mouse is broken"

In [ ]:
# Try the fined tuned model
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role='user', content=the_question)]
)

import textwrap

print ("Using model: " + chat_response.model + ":")
print(textwrap.fill(chat_response.choices[0].message.content, width=80))

In [ ]:
# Try the base Model
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model="open-mistral-7b",
    messages=[ChatMessage(role='user', content=the_question)]
)
print ("Using model: " + chat_response.model + ":")
print(textwrap.fill(chat_response.choices[0].message.content, width=80))